In [2]:
import pandas as pd
import numpy as np
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}


def confusion_matrix(y_true,y_pred):
  y_true=pd.Series([ACTIVITIES[y] for y in np.argmax(y_true,axis=1)])
  y_pred=pd.Series([ACTIVITIES[y] for y in np.argmax(y_pred,axis=1)])

  return pd.crosstab(y_true,y_pred,rownames=['True'], colnames=['Pred'])


### Data

In [4]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [5]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [6]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/MyDrive/appiedAI/HAR/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [7]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/content/drive/MyDrive/appiedAI/HAR/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [8]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [9]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [10]:
# Configuring a session
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [11]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM, BatchNormalization
from keras.layers.core import Dense, Dropout

In [12]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [13]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [14]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [15]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [16]:
model = Sequential()
model.add(LSTM(32, input_shape=(timesteps, input_dim), return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(32))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128, 32)           5376      
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 32)           128       
_________________________________________________________________
dropout (Dropout)            (None, 128, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 198       
Total params: 14,022
Trainable params: 13,958
Non-trainable params: 64
___________________________________

In [17]:
# Compiling the model
model.compile(loss="categorical_crossentropy",
              optimizer="rmsprop",
              metrics=['accuracy'])

In [18]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Epoch 1/30
460/460 [==============================] - 16s 16ms/step - loss: 1.0872 - accuracy: 0.5821 - val_loss: 1.1420 - val_accuracy: 0.6413
Epoch 2/30
460/460 [==============================] - 7s 14ms/step - loss: 0.5434 - accuracy: 0.8011 - val_loss: 0.5990 - val_accuracy: 0.8290
Epoch 3/30
460/460 [==============================] - 6s 14ms/step - loss: 0.3083 - accuracy: 0.9057 - val_loss: 0.3355 - val_accuracy: 0.8880
Epoch 4/30
460/460 [==============================] - 6s 14ms/step - loss: 0.2283 - accuracy: 0.9198 - val_loss: 0.3562 - val_accuracy: 0.8924
Epoch 5/30
460/460 [==============================] - 6s 14ms/step - loss: 0.1955 - accuracy: 0.9363 - val_loss: 0.2835 - val_accuracy: 0.9070
Epoch 6/30
460/460 [==============================] - 7s 14ms/step - loss: 0.1766 - accuracy: 0.9372 - val_loss: 0.3992 - val_accuracy: 0.8941
Epoch 7/30
460/460 [==============================] - 6s 14ms/step - loss: 0.1630 - accuracy: 0.9417 - val_loss: 0.3089 - val_accuracy: 0.898

In [19]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 537        0  ...                   0                 0
SITTING                  0      416  ...                   1                 1
STANDING                 0      104  ...                   0                 0
WALKING                  0        0  ...                  17                 7
WALKING_DOWNSTAIRS       0        0  ...                 397                18
WALKING_UPSTAIRS         0        0  ...                   2               453

[6 rows x 6 columns]


In [20]:
score = model.evaluate(X_test, Y_test)

93/93 [==============================] - 1s 6ms/step - loss: 0.4211 - accuracy: 0.9172


In [21]:
score


[0.42113131284713745, 0.917203962802887]

In [22]:

n_epochs = 30
n_batch = 16
n_classes = _count_classes(Y_train)



In [24]:
model = Sequential()
model.add(LSTM(48, input_shape=(timesteps, input_dim), return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(32))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128, 48)           11136     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 48)           192       
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 48)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                10368     
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 21,894
Trainable params: 21,798
Non-trainable params: 96
_________________________________

In [25]:
# Compiling the model
model.compile(loss="categorical_crossentropy",
              optimizer="rmsprop",
              metrics=['accuracy'])

In [26]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Epoch 1/30
460/460 [==============================] - 10s 15ms/step - loss: 1.0585 - accuracy: 0.5958 - val_loss: 0.7560 - val_accuracy: 0.7628
Epoch 2/30
460/460 [==============================] - 7s 14ms/step - loss: 0.4391 - accuracy: 0.8559 - val_loss: 0.4074 - val_accuracy: 0.8670
Epoch 3/30
460/460 [==============================] - 6s 14ms/step - loss: 0.2686 - accuracy: 0.9143 - val_loss: 0.3710 - val_accuracy: 0.8744
Epoch 4/30
460/460 [==============================] - 7s 15ms/step - loss: 0.2144 - accuracy: 0.9261 - val_loss: 0.2563 - val_accuracy: 0.9080
Epoch 5/30
460/460 [==============================] - 7s 15ms/step - loss: 0.1860 - accuracy: 0.9433 - val_loss: 0.3050 - val_accuracy: 0.9104
Epoch 6/30
460/460 [==============================] - 7s 14ms/step - loss: 0.1674 - accuracy: 0.9353 - val_loss: 0.3348 - val_accuracy: 0.8975
Epoch 7/30
460/460 [==============================] - 7s 14ms/step - loss: 0.1537 - accuracy: 0.9432 - val_loss: 0.3331 - val_accuracy: 0.910

In [27]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 537        0  ...                   0                 0
SITTING                  4      419  ...                   0                 2
STANDING                 0      120  ...                   0                 0
WALKING                  0        0  ...                   6                 5
WALKING_DOWNSTAIRS       0        0  ...                 419                 1
WALKING_UPSTAIRS         0        8  ...                  10               444

[6 rows x 6 columns]


In [28]:
score = model.evaluate(X_test, Y_test)

93/93 [==============================] - 1s 7ms/step - loss: 0.3146 - accuracy: 0.9216


In [29]:
score

[0.3146468698978424, 0.9216151833534241]

In [30]:
from prettytable import PrettyTable
ptable = PrettyTable()
ptable.title = " Model Comparision "
ptable.field_names = ["LSTM Layers",'No. of Neurons in LSTM Layer','Dropout', 'Accuracy']
ptable.add_row(["2","32","0.50","91.72"])
ptable.add_row(["2","48","0.70","92.16"])

print(ptable)

+-----------------------------------------------------------------+
|                        Model Comparision                        |
+-------------+------------------------------+---------+----------+
| LSTM Layers | No. of Neurons in LSTM Layer | Dropout | Accuracy |
+-------------+------------------------------+---------+----------+
|      2      |              32              |   0.50  |  91.72   |
|      2      |              48              |   0.70  |  92.16   |
+-------------+------------------------------+---------+----------+
